# LSH Algorithm Improvement By Applying Bitmap Indexing

In [51]:
import argparse
import sys
from os import listdir
from os.path import isfile, join
from typing import Dict, List, Optional, Tuple
import imagehash
from PIL import Image
import os, os.path
import cv2
from collections import Counter
import scipy as sp
import numpy as np # Import numpy library 
from skimage.feature import hog # Import Hog model to extract features
from sklearn.metrics import confusion_matrix # Import confusion matrix to evaluate the performance
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split

In [34]:
imgs = []
y = []
file_size = []
k = 0
path = "./data/101_ObjectCategories" # Give the dataset path here

##  Data Preprocessing:
1. Load the images using cv2
2. Image resize
3. Feature extraction: BGR to Gray conversion 
4. Feature extraction: Histogram of Oriented Gradients(HOG)

In [35]:
folder = os.listdir(path) # from the given path get the file names such as accordion, airplanes etc..
for file in folder: # for every file name in the given path go inseide that directory and get the images
    subpath = os.path.join(path,file)  # Join the name of these files to the previous path 
    
    files = os.listdir(subpath) # Take these image names to a list called files
    j = 0
    for i in range(np.size(files)): # now we shall loop through these number of files
        
        im = cv2.imread(subpath+'/'+files[0+j]) # Read the images from this subpath
        
        imgs.append(im) # append all the read images to a list called imgs
        y.append(k) # generate a labe to every file and append it to labels list

        j += 1
        if (j == (np.size(files))):
            file_size.append(j)
   
    k += 1
     
y = np.array(y).tolist()
ix = []
for index, item in enumerate(imgs):
    if (np.size(item) == 1):
        ix.append(index)
        del imgs[index]
        
for index, item in enumerate(y):
    for v in range(np.size(ix)):
        if (index == ix[v]):
            del y[index]
        
y = np.array(y).astype(np.float64) 

# Function to convert an image from color to grayscale
def resize_(image):
    u = cv2.resize(image,(256,256))
    return u

def rgb2gray(rgb):
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    return gray

def fd_hog(image):
    fd = hog(image, orientations=8, pixels_per_cell=(64, 64),
                        cells_per_block=(2, 2))
    
    return fd

In [36]:
a=[]
import progressbar
with progressbar.ProgressBar(max_value=len(imgs)) as bar:
    i=1
    for img in imgs:
        b=resize_(img)
        c=rgb2gray(b)   
        d=fd_hog(c)
        a.append(d)
        bar.update(i)
        i+=1
df = pd.DataFrame(a)
df['lable'] = y
id_ = np.arange(1,len(df)+1,1)
df['id'] = id_
X = df.values

spark = SparkSession.builder \
 .master("local") \
 .appName("Image Retrieval") \
 .config("spark.some.config.option", "some-value") \
 .getOrCreate()


100% (9176 of 9176) |####################| Elapsed Time: 0:01:55 Time:  0:01:55


In [37]:
print("HOG diamension: ")
len(a[0])

HOG diamension: 


288

In [140]:
def getBestPerformance(numOfTest, bucketLength,numHashTables,numOfNeighbor):
 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
    Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])
    Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
    Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", 
                                      bucketLength=bucketLength,numHashTables=numHashTables)
    
    
    model = brp.fit(Train_df)
    model.transform(Train_df)
  
    nnToMajorityAccMap = {}
    nnToWeightedAccMap = {}
    with progressbar.ProgressBar(max_value = numOfTest) as bar:
        for i in range(0, numOfTest):
            Catg = X_test[i][-2]
            key = Vectors.dense(X_test[i][0:-2])
            # Choose the Last one of numOfNeighbor, the biggest one 
            result = model.approxNearestNeighbors(Train_df, key, numOfNeighbor[-1], distCol="EuclideanDistance")
            # Conver pySpark framework colunm to python list
            labelList = result.select("label").rdd.flatMap(lambda x: x).collect()
            # Build a distance dataframe for the distance between query and result
            result1 = result.select('label', 'EuclideanDistance').collect()
            df_weighted = pd.DataFrame()
            df_weighted['Label'] = [int(row['label']) for row in result1]
            df_weighted['EuclideanDistance'] = [float(row['EuclideanDistance']) for row in result1]
            df_weighted['Weight'] = 1 / df_weighted['EuclideanDistance']
            # slice LableList into differnt length subLists 
            nnList = []
            for numberNN in numOfNeighbor:
                slicedList = labelList[0:numberNN]
                nnList.append(slicedList)
                
            for index in range(0, len(nnList)):
                majority_vote = Counter(nnList[index]).most_common(1)[0][0]
                if  Catg == majority_vote:
                    key = "M" + str(numOfNeighbor[index])
                    if key in nnToMajorityAccMap:
                        nnToMajorityAccMap[key] = nnToMajorityAccMap.get(key) + 1
                    else:
                        nnToMajorityAccMap[key] = 1
                        
            for n in numOfNeighbor:
                weighted_result = df_weighted.head(n).groupby('Label')['Weight'].sum()
                df_weighted_result = pd.DataFrame(weighted_result)
                df_weighted_result = df_weighted_result.reset_index()
                df_weighted_result.sort_values(by='Weight',inplace=True,ascending=False)
                weighted_vote = df_weighted_result.iloc[0]['Label'].astype('int')
                if  Catg == weighted_vote:
                    key = "W" + str(n)
                    if key in nnToWeightedAccMap:
                        nnToWeightedAccMap[key] = nnToWeightedAccMap.get(key) + 1
                    else:
                        nnToWeightedAccMap[key] = 1
                        
            bar.update(i)
        
        # calculate accuracy base on Majority Vote
        for key in nnToMajorityAccMap:
            nnToMajorityAccMap[key] = nnToMajorityAccMap.get(key) / numOfTest
        # calculate accuracy base on Weighted Vote
        for key in nnToWeightedAccMap:
            nnToWeightedAccMap[key] = nnToWeightedAccMap.get(key) / numOfTest    
    return nnToMajorityAccMap, nnToWeightedAccMap

In [159]:
#set Param 
bucketLengthList = np.arange(20, 61, 10)
numHashTablesList = np.arange(0, 151, 30)
numHashTablesList[0] = 1
#make sure the last element of numOfNeighborList is the largest
numOfNeighborList = [1, 3, 5, 7, 15, 21, 25];
numOfTest = 1000
print("Checking bucketLength Param:")
print(bucketLengthList)
print("Checking numHashTablesList Param:")
print(numHashTablesList)

Checking bucketLength Param:
[20 30 40 50 60]
Checking numHashTablesList Param:
[  1  30  60  90 120 150]


In [160]:
%%time
bucketLengthList_para=[]
numHashTablesList_para=[]
resultMList = []
resultWList = []
for i in bucketLengthList:
    for j in numHashTablesList:
            resultM, resultW = getBestPerformance(numOfTest ,i, j, numOfNeighborList)
            print( "bucketLen:" + str(i) + "  #Hashtable:" + str(j) +  "  Majority Vote Acc: " + str(resultM))
            print( "bucketLen:" + str(i) + "  #Hashtable:" + str(j) +  "  Weighted Vote Acc: " + str(resultW))
            bucketLengthList_para.append(i)
            numHashTablesList_para.append(j)
            resultMList.append(resultM)
            resultWList.append(resultW)

100% (1000 of 1000) |####################| Elapsed Time: 0:26:05 Time:  0:26:05


bucketLen:20  #Hashtable:1  Majority Vote Acc: {'M1': 0.491, 'M3': 0.492, 'M5': 0.484, 'M7': 0.48, 'M15': 0.446, 'M21': 0.434, 'M25': 0.435}
bucketLen:20  #Hashtable:1  Weighted Vote Acc: {'W1': 0.491, 'W3': 0.493, 'W5': 0.488, 'W7': 0.485, 'W15': 0.459, 'W21': 0.443, 'W25': 0.445}


100% (1000 of 1000) |####################| Elapsed Time: 0:24:10 Time:  0:24:10


bucketLen:20  #Hashtable:30  Majority Vote Acc: {'M1': 0.492, 'M3': 0.499, 'M5': 0.504, 'M7': 0.499, 'M15': 0.476, 'M21': 0.476, 'M25': 0.474}
bucketLen:20  #Hashtable:30  Weighted Vote Acc: {'W1': 0.492, 'W3': 0.5, 'W5': 0.507, 'W7': 0.502, 'W15': 0.482, 'W21': 0.48, 'W25': 0.479}


100% (1000 of 1000) |####################| Elapsed Time: 0:34:30 Time:  0:34:30


...

bucketLen:60  #Hashtable:90  Majority Vote Acc: {'M1': 0.506, 'M3': 0.511, 'M5': 0.506, 'M7': 0.504, 'M15': 0.49, 'M21': 0.477, 'M25': 0.466}
bucketLen:60  #Hashtable:90  Weighted Vote Acc: {'W1': 0.506, 'W3': 0.512, 'W5': 0.515, 'W7': 0.511, 'W15': 0.496, 'W21': 0.484, 'W25': 0.471}


100% (1000 of 1000) |####################| Elapsed Time: 0:41:37 Time:  0:41:37


bucketLen:60  #Hashtable:120  Majority Vote Acc: {'M1': 0.506, 'M3': 0.508, 'M5': 0.503, 'M7': 0.502, 'M15': 0.486, 'M21': 0.469, 'M25': 0.46}
bucketLen:60  #Hashtable:120  Weighted Vote Acc: {'W1': 0.506, 'W3': 0.512, 'W5': 0.507, 'W7': 0.511, 'W15': 0.492, 'W21': 0.478, 'W25': 0.473}


100% (1000 of 1000) |####################| Elapsed Time: 1:31:53 Time:  1:31:53


bucketLen:60  #Hashtable:150  Majority Vote Acc: {'M1': 0.493, 'M3': 0.491, 'M5': 0.498, 'M7': 0.493, 'M15': 0.49, 'M21': 0.469, 'M25': 0.465}
bucketLen:60  #Hashtable:150  Weighted Vote Acc: {'W1': 0.493, 'W3': 0.493, 'W5': 0.502, 'W7': 0.498, 'W15': 0.496, 'W21': 0.476, 'W25': 0.473}
CPU times: user 22min 5s, sys: 2min 51s, total: 24min 56s
Wall time: 18h 23min 49s


In [161]:
df_result = pd.DataFrame()
df_result['BucketLength'] = bucketLengthList_para
df_result['NumHashTables'] = numHashTablesList_para
# conver List of map to panda dataframework
df_result_map =  pd.DataFrame(resultMList)
df_result_map2 = pd.DataFrame(resultWList)
df_result = pd.concat([df_result, df_result_map], axis=1, join='inner')
df_result = pd.concat([df_result, df_result_map2], axis=1, join='inner')

df_result.to_csv('./result3.csv') #Chang the name every you wanna sava a file

In [162]:
df_result

,BucketLength,NumHashTables,M1,M15,M21,M25,M3,M5,M7,W1,W15,W21,W25,W3,W5,W7
0,20,1,0.491,0.446,0.434,0.435,0.492,0.484,0.480,0.491,0.459,0.443,0.445,0.493,0.488,0.485
1,20,30,0.492,0.476,0.476,0.474,0.499,0.504,0.499,0.492,0.482,0.480,0.479,0.500,0.507,0.502
2,20,60,0.515,0.513,0.502,0.497,0.515,0.520,0.506,0.515,0.519,0.507,0.503,0.516,0.523,0.510
3,20,90,0.497,0.479,0.468,0.468,0.498,0.488,0.488,0.497,0.486,0.477,0.471,0.501,0.495,0.495
4,20,120,0.492,0.462,0.444,0.439,0.483,0.481,0.479,0.492,0.469,0.450,0.444,0.483,0.484,0.487
5,20,150,0.507,0.486,0.472,0.463,0.508,0.503,0.504,0.507,0.496,0.484,0.473,0.510,0.508,0.509
6,30,1,0.468,0.445,0.429,0.431,0.469,0.469,0.464,0.468,0.451,0.438,0.440,0.468,0.468,0.465
7,30,30,0.501,0.480,0.464,0.459,0.509,0.499,0.499,0.501,0.483,0.468,0.465,0.511,0.503,0.502
8,30,60,0.503,0.481,0.474,0.467,0.515,0.519,0.516,0.503,0.486,0.478,0.473,0.515,0.521,0.520
9,30,90,0.534,0.519,0.506,0.488,0.533,0.531,0.535,0.534,0.522,0.510,0.494,0.533,0.532,0.540


# !!!Skip all the code below !!!

## Split data
Split the data to training and validation data. We choose 70% for training and 30% for validation purposes.

In [6]:
%%time
# append 'label' and 'id' to the last two colunms
df = pd.DataFrame(a)
df['lable'] = y
id_ = np.arange(1,len(df)+1,1)
df['id'] = id_
X = df.values

CPU times: user 1.74 s, sys: 428 ms, total: 2.16 s
Wall time: 2.97 s


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Using PySpark to retrieve similar images

In [ ]:
spark = SparkSession.builder \
     .master("local") \
     .appName("Image Retrieval") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [8]:
Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])

In [9]:
Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

In [10]:
Train_df.show(n = 2)

+----+-----+--------------------+
|  id|label|            features|
+----+-----+--------------------+
|5960|   80|[0.00363684489770...|
|1536|   18|[0.06071563370892...|
+----+-----+--------------------+
only showing top 2 rows



In [11]:
%reload_ext memory_profiler
def toMeasureMemo():
    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes",bucketLength=2,numHashTables=3)
    model = brp.fit(Train_df)
    print("The hashed dataset where hashed values are stored in the column 'hashes':")
    model.transform(Train_df).show()
%memit  toMeasureMemo()

The hashed dataset where hashed values are stored in the column 'hashes':
+----+-----+--------------------+--------------------+
|  id|label|            features|              hashes|
+----+-----+--------------------+--------------------+
|5960|   80|[0.00363684489770...|[[0.0], [0.0], [-...|
|1536|   18|[0.06071563370892...|[[0.0], [0.0], [-...|
|8387|   95|[0.01072387506065...|[[-1.0], [0.0], [...|
|6169|   83|[0.00663216869878...|[[0.0], [0.0], [0...|
|4067|   55|[0.01922287915132...|[[-1.0], [-1.0], ...|
|2135|   23|[0.02042261440886...|[[0.0], [0.0], [0...|
|8246|   95|[0.02765804852049...|[[-1.0], [-1.0], ...|
|8690|   96|[0.03719083421779...|[[-1.0], [0.0], [...|
|6724|   88|[0.06854134811831...|[[-1.0], [0.0], [...|
|  26|    0|[0.04798466879403...|[[-1.0], [0.0], [...|
|6432|   88|[0.01443607483278...|[[0.0], [0.0], [-...|
|2392|   28|[0.02645665091604...|[[-1.0], [0.0], [...|
|1580|   18|[0.16710970919873...|[[-1.0], [0.0], [...|
|1388|   18|[0.04740743977419...|[[0.0], [0.0]

In [ ]:
key = Vectors.dense(X_test[0][0:-2])

In [ ]:
key

In [ ]:
X_test[0][-2]

In [ ]:
print("Approximately searching Train_df for 2 nearest neighbors of the key:")
model.approxNearestNeighbors(Train_df, key, 5).show()

In [ ]:
# result_id = result.select('label',).collect()
# result_id[0].label

In [ ]:
# print("Approximately joining Train_df and Test_df on Euclidean distance smaller than 1:")
# model.approxSimilarityJoin(Train_df, Test_df, 1.1, distCol="EuclideanDistance")\
#     .select(col("datasetA.id").alias("Train_df"),
#             col("datasetB.id").alias("Test_df"),
#             col("EuclideanDistance")).show(30)

In [ ]:
accuracy = 0
numOfNeighbor = 5
numOfTest= 5
accList = []
with progressbar.ProgressBar(max_value=numOfTest) as bar:
    for i in range(0, numOfTest):
        Catg = X_test[i][-2]
        key = Vectors.dense(X_test[i][0:-2])
        result = model.approxNearestNeighbors(Train_df, key, numOfNeighbor)
        temp = Counter([int(row['label']) for row in result.collect()])
        if  Catg in temp:
            accuracy += temp.get(Catg)/ numOfNeighbor
            accList.append(temp.get(Catg)/ numOfNeighbor)
        else:
            accList.append(0)
        bar.update(i)
    accuracy /= numOfTest

In [ ]:
accuracy

In [ ]:
print(accList)

In [ ]:
# from matplotlib.pyplot import imshow
# imshow(imgs[4795])

In [32]:
%load_ext memory_profiler
def my_func():
    a = [1] * (10 **6)
    b = [2] * (2 * 10 ** 7)
    del b
    return a
%memit my_func()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 288.71 MiB, increment: 160.33 MiB


[[1], [1, 2, 3], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6, 7]]